In [ ]:
import numpy as np
import h5py
import setGPU

import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Lambda, BatchNormalization, Activation, Concatenate, Dropout, Layer
from tensorflow.keras.layers import ReLU, LeakyReLU
from tensorflow.keras import backend as K
import math
import pickle

from datetime import datetime
from tensorboard import program
import os
import tensorflow_model_optimization as tfmot
from qkeras import QDense, QActivation
from qkeras import *

import matplotlib.pyplot as plt
import matplotlib
%matplotlib inline

from functions import preprocess_anomaly_data, custom_loss_negative, custom_loss_training,\
roc_objective,load_model, save_model
from custom_layers import Sampling

from autoencoder_classes import VAE

tsk = tfmot.sparsity.keras
# from tensorflow.python.client import device_lib 
# print(device_lib.list_local_devices())

In [ ]:
# Data = (N,19,3,1).flatten()
with open('/eos/user/e/epuljak/forDelphes/Delphes_QCD_BSM_data.pkl', 'rb') as f:
    X_train_flatten, X_train_scaled, X_test_flatten, X_test_scaled, bsm_data, bsm_target, pt_scaler = pickle.load(f)

In [ ]:
quant_size = 8
integer = 3
symmetric = 1
alpha=1

### Define model
Prune and quantize only encoder.

In [ ]:
latent_dim = 3
input_shape = 57

In [ ]:
#encoder
inputArray = Input(shape=(input_shape))
#proba
x = QActivation(f'quantized_bits(16,10,1,alpha=1)')(inputArray)
x = BatchNormalization()(x)
x = Dense(32, kernel_initializer=tf.keras.initializers.HeNormal(seed=42))(x) if quant_size==0\
    else QDense(32, kernel_initializer=tf.keras.initializers.HeNormal(seed=42),\
               kernel_quantizer=f'quantized_bits(bits=' + str(quant_size) + ', integer=' + str(integer) + ', symmetric=1, alpha=1)',\
               bias_quantizer=f'quantized_bits(bits=' + str(quant_size) + ', integer=' + str(integer) + ', symmetric=1, alpha=1)')(x)
x = BatchNormalization()(x)
x = Activation('relu')(x) if quant_size==0\
    else QActivation(f'quantized_relu(bits=' + str(quant_size) + ', integer=' + str(integer) + ')')(x)
x = Dense(16, kernel_initializer=tf.keras.initializers.HeNormal(seed=42))(x) if quant_size==0\
    else QDense(16, kernel_initializer=tf.keras.initializers.HeNormal(seed=42),\
               kernel_quantizer=f'quantized_bits(bits=' + str(quant_size) + ', integer=' + str(integer) + ', symmetric=1, alpha=1)',\
               bias_quantizer=f'quantized_bits(bits=' + str(quant_size) + ', integer=' + str(integer) + ', symmetric=1, alpha=1)')(x)
x = BatchNormalization()(x)
x = Activation('relu')(x) if quant_size==0\
    else QActivation(f'quantized_relu(bits=' + str(quant_size) + ', integer=' + str(integer) + ')')(x)
mu = Dense(latent_dim, name = 'latent_mu', kernel_initializer=tf.keras.initializers.HeNormal(seed=42))(x) if quant_size==0\
    else QDense(latent_dim, kernel_initializer=tf.keras.initializers.HeNormal(seed=42),\
               kernel_quantizer=f'quantized_bits(bits=' + str(16) + ', integer=' + str(6) + ', symmetric=1, alpha=1)',\
               bias_quantizer=f'quantized_bits(bits=' + str(16) + ', integer=' + str(6) + ', symmetric=1, alpha=1)')(x)
logvar = Dense(latent_dim, name = 'latent_logvar', kernel_initializer=tf.keras.initializers.HeNormal(seed=42))(x) if quant_size==0\
    else QDense(latent_dim, kernel_initializer=tf.keras.initializers.HeNormal(seed=42),\
               kernel_quantizer=f'quantized_bits(bits=' + str(16) + ', integer=' + str(6) + ', symmetric=1, alpha=1)',\
               bias_quantizer=f'quantized_bits(bits=' + str(16) + ', integer=' + str(6) + ', symmetric=1, alpha=1)')(x)
# Use reparameterization trick to ensure correct gradient
z = Sampling()([mu, logvar])

# Create encoder
encoder = Model(inputArray, [mu, logvar, z], name='encoder')    
encoder.summary()


#decoder
d_input = Input(shape=(latent_dim,), name='decoder_input')
x = Dense(16, kernel_initializer=tf.keras.initializers.HeNormal(seed=42))(d_input)
x = BatchNormalization()(x)
#x = LeakyReLU(alpha=0.3)(x)
x = Activation('relu')(x)
x = Dense(32, kernel_initializer=tf.keras.initializers.HeNormal(seed=42))(x)    
x = BatchNormalization()(x)
#x = LeakyReLU(alpha=0.3)(x)
x = Activation('relu')(x)
dec = Dense(input_shape, kernel_initializer=tf.keras.initializers.HeNormal(seed=42))(x)
# Create decoder
decoder = Model(d_input, dec, name='decoder')
decoder.summary()

In [ ]:
vae = VAE(encoder, decoder)
vae.compile(optimizer=keras.optimizers.Adam())

In [ ]:
# load BP or higher bit width model
model_dir = 'VAE_models/final_models/withCorrectPrefiltering/'
name_encoder ='VAE_encoder_pruned'
name_decoder ='VAE_decoder_pruned'
custom_objects={'Sampling': Sampling}

BP_encoder = load_model(model_dir+name_encoder, custom_objects)
BP_decoder = load_model(model_dir+name_decoder, custom_objects)

#BP_encoder, BP_decoder = VAE.load(model_dir, custom_objects)

In [ ]:
# set weights for encoder
for i, l in enumerate(vae.encoder.layers):
    if i < 2: continue
    vae.encoder.layers[i].set_weights(BP_encoder.layers[i-1].get_weights()) # i-1 because of QActivation layer (remove when loading from qkeras model)

In [ ]:
# set weights for encoder
#vae.decoder.load_weights('/eos/user/e/epuljak/autoencoder_models/VAE_decoder_pruned.h5')
for i, l in enumerate(vae.decoder.layers):
    if i == 0: continue
    vae.decoder.layers[i].set_weights(BP_decoder.layers[i].get_weights())

In [ ]:
# check quantizers
for layer in vae.encoder.layers:
    if hasattr(layer, "kernel_quantizer"):
        print(layer.name, "kernel:", str(layer.kernel_quantizer_internal)," scale:",str(layer.kernel_quantizer_internal.scale), "bias:", str(layer.bias_quantizer_internal))
    elif hasattr(layer, "quantizer"):
        print(layer.name, "quantizer:", str(layer.quantizer))

## Check range of mean and logvar in latent space

In [ ]:
mean, logvar, _ = vae.encoder.predict(X_test_flatten[:10000])

plt.hist(logvar[:,0], bins=100, range=(-0.2,0.2))
plt.show()

plt.hist(logvar[:,1], bins=100, range=(-0.2,0.2))
plt.show()
plt.hist(logvar[:,2], bins=100, range=(-0.2,0.2))
plt.show()

## Add custom KL layer

In [ ]:
from custom_layers import KLLoss

In [ ]:
# take only encoder
#model = final_encoder
# get mu and sigma from model
z_mean = vae.encoder.layers[-3].output
z_log_var = vae.encoder.layers[-2].output
# calculate KL distance with the custom layer
custom_output = KLLoss()([z_mean, z_log_var])
# create new model
model = Model(inputs=vae.encoder.input, outputs=custom_output)
model.summary()

In [ ]:
#keras.utils.plot_model(model, show_shapes=True)

In [ ]:
#model.compile()

In [ ]:
#save_model('VAE_models/final_models/withCorrectPrefiltering/VAE_encoder_PTQ', model)


model = load_model('VAE_models/final_models/withCorrectPrefiltering/VAE_encoder_PTQ', custom_objects={'KLLoss': KLLoss, 'QActivation': QActivation, 'QDense': QDense})

## Convert to HLS

In [ ]:
def check_diff_layer_by_layer(layer_keras, layer_hls):
    print(f'Keras layer {layer_hls}, first sample:')
    print(config['LayerName'][layer_hls])
    print(keras_trace[layer_keras][:].flatten()[:])
    print(hls4ml_trace[layer_hls][:].flatten()[:])
    print(keras_trace[layer_keras][:].flatten()[:]-hls4ml_trace[layer_hls][:].flatten()[:])
    plt.hist(keras_trace[layer_keras][:].flatten()[:]-hls4ml_trace[layer_hls][:].flatten()[:], bins=100, range=(-1,1))
    plt.title(layer_hls)
    plt.show()

In [ ]:
import hls4ml

hls4ml.model.optimizer.OutputRoundingSaturationMode.layers = ['Activation', 'KLLoss']
hls4ml.model.optimizer.OutputRoundingSaturationMode.rounding_mode = 'AP_RND_CONV'
hls4ml.model.optimizer.OutputRoundingSaturationMode.saturation_mode = 'AP_SAT'

In [ ]:
hardware = 'xcvu9p-flgb2104-2-e'


In [ ]:
config = hls4ml.utils.config_from_keras_model(model, default_precision='ap_fixed<16,6,AP_RND_CONV,AP_SAT>',
        max_bits=20,
        data_type_mode='auto_accum', # auto_accum_only
        granularity='name')

In [ ]:
# if you want to skip merging dense+batchnorm layer in hls
config['SkipOptimizers'] = ['fuse_batch_norm']

In [ ]:
config['Model']['Strategy'] = 'Resource'

In [ ]:
# update config
config['LayerName']['kl_loss'].update({
        'Precision': {
            'accum': 'ap_fixed<32,10,AP_RND,AP_SAT>',
            'result': 'ap_fixed<32,10,AP_RND,AP_SAT>'
        },
        'sum_t': 'ap_fixed<32,10>',
        'exp_range': 0.5,
        'exp_table_t': 'ap_fixed<32,10,AP_RND,AP_SAT>',
        'table_size': 1024*4
    })

In [ ]:
config['LayerName']['input_1'].update({
        'Precision': 'ap_fixed<16,10>'
        })

In [ ]:
config['LayerName']['latent_mu']['Precision'].update({
                'result': 'ap_fixed<32,6>'
            })
config['LayerName']['latent_logvar']['Precision'].update({
        'result': 'ap_fixed<32,6>'
    })
config['LayerName']['latent_mu_linear'] = {
        'Precision': 'ap_fixed<32, 6, AP_RND_CONV, AP_SAT>'
    }
config['LayerName']['latent_logvar_linear'] = {
        'Precision': 'ap_fixed<32, 6, AP_RND_CONV, AP_SAT>'
    }

In [ ]:
config['LayerName']['batch_normalization'].update({
               'Precision': {'scale': 'ap_fixed<16,7>',
                             'bias': 'ap_fixed<18,3>',
                            'result': 'ap_fixed<16,6>'}
                
})

In [ ]:
config['LayerName']['q_dense'].update({
               'Precision': {'result': 'ap_fixed<14,4>',
                                'weight': 'ap_fixed<12,5>',
                                'bias': 'ap_fixed<12,5>',
                                'accum': 'ap_fixed<32,16,AP_RND,AP_SAT>'}
                
})

In [ ]:
config['LayerName']['q_dense_1'].update({
               'Precision': {'result': 'ap_fixed<16,6>',
                                'weight': 'ap_fixed<12,5>',
                                'bias': 'ap_fixed<12,5>',
                                'accum': 'ap_fixed<32,16,AP_RND,AP_SAT>'}
                
})

In [ ]:
config['LayerName']['batch_normalization_1'].update({
               'Precision': {'scale': 'ap_fixed<3,2>',
                                'bias': 'ap_fixed<8,1>',
                                'result': 'ap_fixed<16,6,AP_RND_CONV,AP_SAT>'}
                
})

In [ ]:
# set precision for mean layer
config['LayerName']['q_dense_2'].update({
               'Precision': {'result': 'ap_fixed<16,6>',
                            'weight': 'ap_fixed<16,7>',
                            'bias': 'ap_fixed<16,7>',
                            'accum': 'ap_fixed<32,8,AP_RND,AP_SAT>'}
                
})

In [ ]:
# set precision for logvar layer

config['LayerName']['q_dense_3'].update({
               'Precision': {'result': 'ap_fixed<16,6>',
                            'weight': 'ap_fixed<16,7>',
                            'bias': 'ap_fixed<16,7>',
                            'accum': 'ap_fixed<32,8,AP_RND,AP_SAT>'}
                
})

In [ ]:
from hls4ml.utils.config import set_accum_from_keras_model
set_accum_from_keras_model(config, model)

In [ ]:
# save config
output_config = 'VAE_models/final_models/withCorrectPrefiltering/VAE_config_HLS.pkl'
# with open(output_config, 'wb') as handle:
#     pickle.dump(config, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open(output_config, 'rb') as handle:
    config = pickle.load(handle)

In [ ]:
hls_model = hls4ml.converters.convert_from_keras_model(model,
                                                       hls_config=config,
                                                       output_dir='output/DVAE_PTQ/xcvu9p/',
                                                       fpga_part=hardware)

In [ ]:
hls4ml.model.profiling.numerical(model=model, hls_model=hls_model, X=X_test_flatten[:100000])
hls4ml.utils.plot_model(hls_model, show_shapes=True, show_precision=True, to_file='ptq_VAE_qkeras_%d.pdf'%quant_size)

## TRACE

In [ ]:
for layer in config['LayerName'].keys():
    config['LayerName'][layer]['Trace'] = True
hls_model = hls4ml.converters.convert_from_keras_model(model,
                                                       hls_config=config,
                                                       output_dir='output/DVAE_PTQ/xcvu9p/',
                                                       fpga_part=hardware)

In [ ]:
hls_model.compile()

In [ ]:
model.compile() #after you compile hls model

In [ ]:
hls4ml_pred, hls4ml_trace = hls_model.trace(X_test_flatten[:100000])
keras_trace = hls4ml.model.profiling.get_ymodel_keras(model, X_test_flatten[:100000])

In [ ]:
# check
hls4ml_trace.keys()

In [ ]:
# check
keras_trace.keys()

In [ ]:
for layer in hls4ml_trace.keys():
    if layer in keras_trace.keys():
        print(f'Keras layer {layer}, first sample:')
        print(config['LayerName'][layer])
        print(keras_trace[layer][:].flatten()[:])
        print(hls4ml_trace[layer][:].flatten()[:])
        print(keras_trace[layer][:].flatten()[:]-hls4ml_trace[layer][:].flatten()[:])
        plt.hist(keras_trace[layer][:].flatten()[:]-hls4ml_trace[layer][:].flatten()[:], bins=100, range=(-1,1))
        plt.title(layer)
        plt.show()

In [ ]:
check_diff_layer_by_layer('latent_mu_function', 'latent_mu_linear')
check_diff_layer_by_layer('latent_logvar_function', 'latent_logvar_linear')

In [ ]:
for layer in hls4ml_trace.keys():
    plt.figure()
    klayer = layer
    if '_alpha' in layer:
        klayer = layer.replace('_alpha','')
    plt.scatter(hls4ml_trace[layer].flatten(), keras_trace[klayer].flatten(), s=0.2)
    min_x = min(np.amin(hls4ml_trace[layer]), np.amin(keras_trace[klayer]))
    max_x = max(np.amax(hls4ml_trace[layer]), np.amax(keras_trace[klayer]))
    plt.plot([min_x, max_x], [min_x, max_x], c='gray')
    plt.xlabel('hls4ml {}'.format(layer))
    plt.ylabel('QKeras {}'.format(klayer))
    plt.show()
    #plt.savefig('profiling_{}.png'.format(layer), dpi=300)

In [ ]:
layer = 'batch_normalization'
print(f'Keras layer batch_normalization, first sample:')
#print(config['LayerName'][layer])
print(keras_trace[layer][:].flatten()[:])
print(hls4ml_trace[layer][:].flatten()[:])
print(keras_trace[layer][:].flatten()[:]-hls4ml_trace[layer][:].flatten()[:])
plt.hist(keras_trace[layer][:].flatten()[:]-hls4ml_trace[layer][:].flatten()[:], bins=100, range=(-1,1))
plt.title(layer)
plt.show()

layer = 'dense'
print(f'Keras layer dense, first sample:')
#print(config['LayerName'][layer])
print(keras_trace['batch_normalization_1'][:].flatten()[:])
print(hls4ml_trace[layer][:].flatten()[:])
print(keras_trace['batch_normalization_1'][:].flatten()[:]-hls4ml_trace[layer][:].flatten()[:])
plt.hist(keras_trace['batch_normalization_1'][:].flatten()[:]-hls4ml_trace[layer][:].flatten()[:], bins=100, range=(-1,1))
plt.title(layer)
plt.show()

layer = 'leaky_re_lu'
print(f'Keras layer leaky_re_lu, first sample:')
#print(config['LayerName'][layer])
print(keras_trace[layer][:].flatten()[:])
print(hls4ml_trace[layer][:].flatten()[:])
print(keras_trace[layer][:].flatten()[:]-hls4ml_trace[layer][:].flatten()[:])
plt.hist(keras_trace[layer][:].flatten()[:]-hls4ml_trace[layer][:].flatten()[:], bins=100, range=(-1,1))
plt.title(layer)
plt.show()

layer = 'dense_1'
print(f'Keras layer dense, first sample:')
#print(config['LayerName'][layer])
print(keras_trace['batch_normalization_2'][:].flatten()[:])
print(hls4ml_trace[layer][:].flatten()[:])
print(keras_trace['batch_normalization_2'][:].flatten()[:]-hls4ml_trace[layer][:].flatten()[:])
plt.hist(keras_trace['batch_normalization_2'][:].flatten()[:]-hls4ml_trace[layer][:].flatten()[:], bins=100, range=(-1,1))
plt.title(layer)
plt.show()

layer = 'leaky_re_lu_1'
print(f'Keras layer leaky_re_lu, first sample:')
#print(config['LayerName'][layer])
print(keras_trace[layer][:].flatten()[:])
print(hls4ml_trace[layer][:].flatten()[:])
print(keras_trace[layer][:].flatten()[:]-hls4ml_trace[layer][:].flatten()[:])
plt.hist(keras_trace[layer][:].flatten()[:]-hls4ml_trace[layer][:].flatten()[:], bins=100, range=(-1,1))
plt.title(layer)
plt.show()

layer = 'latent_mu'
print(f'Keras layer leaky_re_lu, first sample:')
#print(config['LayerName'][layer])
print(keras_trace[layer][:].flatten()[:])
print(hls4ml_trace[layer][:].flatten()[:])
print(keras_trace[layer][:].flatten()[:]-hls4ml_trace[layer][:].flatten()[:])
plt.hist(keras_trace[layer][:].flatten()[:]-hls4ml_trace[layer][:].flatten()[:], bins=100, range=(-1,1))
plt.title(layer)
plt.show()

layer = 'latent_logvar'
print(f'Keras layer leaky_re_lu, first sample:')
#print(config['LayerName'][layer])
print(keras_trace[layer][:].flatten()[:])
print(hls4ml_trace[layer][:].flatten()[:])
print(keras_trace[layer][:].flatten()[:]-hls4ml_trace[layer][:].flatten()[:])
plt.hist(keras_trace[layer][:].flatten()[:]-hls4ml_trace[layer][:].flatten()[:], bins=100, range=(-1,1))
plt.title(layer)
plt.show()

layer = 'kl_loss'
print(f'Keras layer leaky_re_lu, first sample:')
#print(config['LayerName'][layer])
print(keras_trace[layer][:].flatten()[:])
print(hls4ml_trace[layer][:].flatten()[:])
print(keras_trace[layer][:].flatten()[:]-hls4ml_trace[layer][:].flatten()[:])
plt.hist(keras_trace[layer][:].flatten()[:]-hls4ml_trace[layer][:].flatten()[:], bins=100, range=(-1,1))
plt.title(layer)
plt.show()

In [ ]:
# remove linear nodes from the network
for l in list(hls_model.get_layers()):
    if '_linear' in l.name: hls_model.remove_node(l)

In [ ]:
hls4ml.model.profiling.compare(model, hls_model, X_test_flatten[:100000], 'dist_diff')

In [ ]:
hls4ml.model.profiling.compare(model, hls_model, X_test_flatten[:1000000], 'norm_diff')

## CHECK ROCs Keras vs HLS model

In [ ]:
# keras model predictions
y = model.predict(X_test_flatten)

In [ ]:
# HLS model predictions
y_hls = hls_model.predict(X_test_flatten)

In [ ]:
# for KL layer output
kl_loss_total = []
kl_loss_total.append(y)
kl_loss_total.append(y_hls)

In [ ]:
bsm_labels = ['Leptoquark','A to 4 leptons', 'hChToTauNu', 'hToTauTau']
labels = ['QCD keras', 'QCD hls',\
          r'QKeras LQ $\rightarrow$ b$\tau$', r'HLS LQ $\rightarrow$ b$\tau$',\
          r'QKeras A $\rightarrow$ 4L', r'HLS A $\rightarrow$ 4L',\
          r'QKeras $h_{\pm} \rightarrow \tau\nu$', r'HLS $h_{\pm} \rightarrow \tau\nu$',\
          r'QKeras $h_{0} \rightarrow \tau\tau$', r'HLS $h_{0} \rightarrow \tau\tau$']
loss = '$D_{KL}$'

In [ ]:
for i, label in enumerate(bsm_labels):
    hls4ml_pred = hls_model.predict(bsm_data[i])
    keras_pred = model.predict(bsm_data[i])
    
    kl_loss_total.append(keras_pred)
    kl_loss_total.append(hls4ml_pred)
    print("========================================================================")

In [ ]:
minScore = 999999.
maxScore = 0
for i in range(len(labels)):
    thisMin = np.min(kl_loss_total[i])
    thisMax = np.max(kl_loss_total[i])
    minScore = min(thisMin, minScore)
    maxScore = max(maxScore, thisMax)

In [ ]:
colors = ['C1','C2', 'C3', 'C4', 'C5', 'C6']

In [ ]:
bin_size=100
plt.figure(figsize=(10,8))
z = 0
for i, label in enumerate(labels):
    if i%2==0:
        plt.hist(kl_loss_total[i].reshape(kl_loss_total[i].shape[0]*1), bins=bin_size, label=label, density = True, range=(minScore, maxScore),
         histtype='step', fill=False, linewidth=1.5, color=colors[z])
    if i%2==1:
        plt.hist(kl_loss_total[i].reshape(kl_loss_total[i].shape[0]*1), bins=bin_size, label=label, density = True, range=(minScore, maxScore),
         histtype='step', fill=False, linewidth=1.5, alpha=0.6, color=colors[z])
        z = z+1
#plt.semilogx()
plt.semilogy()
plt.xlabel("Loss")
plt.ylabel("Probability (a.u.)")
plt.grid(True)
plt.title('KL loss')
plt.legend(loc='best')
plt.show()

In [ ]:
from sklearn.metrics import roc_curve, auc
tpr_lq=[];fpr_lq=[];auc_lq=[]
tpr_ato4l=[];fpr_ato4l=[];auc_ato4l=[]
tpr_ch=[];fpr_ch=[];auc_ch=[]
tpr_to=[];fpr_to=[];auc_to=[]


target_qcd = np.zeros(kl_loss_total[0].shape[0])
target_qcd_hls = np.zeros(kl_loss_total[1].shape[0])

for i, label in enumerate(labels):
    if i == 0 and i==1: continue
    if i%2==0:
        trueVal = np.concatenate((np.ones(kl_loss_total[i].shape[0]), target_qcd))
        predVal_loss = np.concatenate((kl_loss_total[i], kl_loss_total[0]))

        fpr_loss, tpr_loss, threshold_loss = roc_curve(trueVal, predVal_loss)

        auc_loss = auc(fpr_loss, tpr_loss)
        if i==2:
            tpr_lq.append(tpr_loss)
            fpr_lq.append(fpr_loss)
            auc_lq.append(auc_loss)
        elif i == 4:
            tpr_ato4l.append(tpr_loss)
            fpr_ato4l.append(fpr_loss)
            auc_ato4l.append(auc_loss)
        elif i==6:
            tpr_ch.append(tpr_loss)
            fpr_ch.append(fpr_loss)
            auc_ch.append(auc_loss)
        elif i == 8:
            tpr_to.append(tpr_loss)
            fpr_to.append(fpr_loss)
            auc_to.append(auc_loss)
    if i%2==1:
        
        trueVal = np.concatenate((np.ones(kl_loss_total[i].shape[0]), target_qcd_hls))
        predVal_loss = np.concatenate((kl_loss_total[i], kl_loss_total[1]))

        fpr_loss, tpr_loss, threshold_loss = roc_curve(trueVal, predVal_loss)

        auc_loss = auc(fpr_loss, tpr_loss)
        if i==3:
            tpr_lq.append(tpr_loss)
            fpr_lq.append(fpr_loss)
            auc_lq.append(auc_loss)
        elif i == 5:
            tpr_ato4l.append(tpr_loss)
            fpr_ato4l.append(fpr_loss)
            auc_ato4l.append(auc_loss)
        elif i==7:
            tpr_ch.append(tpr_loss)
            fpr_ch.append(fpr_loss)
            auc_ch.append(auc_loss)
        elif i == 9:
            tpr_to.append(tpr_loss)
            fpr_to.append(fpr_loss)
            auc_to.append(auc_loss)

In [ ]:
plt.figure(figsize=(12,8))
for i, (tpr, fpr, auc, L) in enumerate(zip(tpr_lq[:], fpr_lq[:], auc_lq[:], labels[2:4])):
    if i == 1:
        plt.plot(fpr, tpr, "-", label='%s (auc = %.1f%%)'%(L,auc*100.), linewidth=1.5, color=colors[0], alpha=0.6, linestyle='dashed')
    else: 
        plt.plot(fpr, tpr, "-", label='%s (auc = %.1f%%)'%(L,auc*100.), linewidth=1.5, color=colors[0])

for i, (tpr, fpr, auc, L) in enumerate(zip(tpr_ato4l[:], fpr_ato4l[:], auc_ato4l[:], labels[4:6])):
    if i == 1: plt.plot(fpr, tpr, "-", label='%s (auc = %.1f%%)'%(L,auc*100.), linewidth=1.5, color=colors[1], alpha = 0.6, linestyle='dashed')
    else: plt.plot(fpr, tpr, "-", label='%s (auc = %.1f%%)'%(L,auc*100.), linewidth=1.5, color=colors[1])
for i, (tpr, fpr, auc, L) in enumerate(zip(tpr_ch[:], fpr_ch[:], auc_ch[:], labels[6:8])):
    if i==1: plt.plot(fpr, tpr, "-", label='%s (auc = %.1f%%)'%(L,auc*100.), linewidth=1.5, color=colors[2], alpha=0.6, linestyle='dashed')
    else: plt.plot(fpr, tpr, "-", label='%s (auc = %.1f%%)'%(L,auc*100.), linewidth=1.5, color=colors[2])

for i, (tpr, fpr, auc, L) in enumerate(zip(tpr_to[:], fpr_to[:], auc_to[:], labels[8:])):
    if i==1: plt.plot(fpr, tpr, "-", label='%s (auc = %.1f%%)'%(L,auc*100.), linewidth=1.5, color=colors[3], alpha=0.6, linestyle='dashed')
    else: plt.plot(fpr, tpr, "-", label='%s (auc = %.1f%%)'%(L,auc*100.), linewidth=1.5, color=colors[3])
plt.semilogx()
plt.semilogy()
plt.ylabel("True Positive Rate", fontsize=15)
plt.xlabel("False Positive Rate", fontsize=15)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.grid(True)
plt.legend(bbox_to_anchor=[1.2, 0.5],loc='best',frameon=True)
plt.tight_layout()
plt.plot(np.linspace(0, 1),np.linspace(0, 1), '--', color='0.75')
plt.axvline(0.00001, color='red', linestyle='dashed', linewidth=1)
plt.show()

# Check ROCs QKeras vs BP model

In [ ]:
with open('/eos/user/e/epuljak/forDelphes/Delphes_QCD_BSM_data_half2.pkl', 'rb') as f:
    X_train_flatten, X_train_scaled, X_test_flatten, X_test_scaled, bsm_data, bsm_target = pickle.load(f)

In [ ]:
## create BP model with KL layer

# get mu and sigma from model
z_mean = BP_encoder.layers[-3].output
z_log_var = BP_encoder.layers[-2].output
# calculate KL distance with the custom layer
custom_output = KLLoss()([z_mean, z_log_var])
# create new model
BP_model = Model(inputs=BP_encoder.input, outputs=custom_output)
BP_model.summary()
BP_model.compile()

In [ ]:
y_keras = model.predict(X_test_flatten)
y_BP = BP_model.predict(X_test_flatten)

In [ ]:
kl_loss_total = []
kl_loss_total.append(y_keras)
kl_loss_total.append(y_BP)

In [ ]:
bsm_labels = ['Leptoquark','A to 4 leptons', 'hChToTauNu', 'hToTauTau']
labels = ['QCD QKeras', 'QCD BP Keras',\
          r'QKeras LQ $\rightarrow$ b$\tau$', r'BP Keras LQ $\rightarrow$ b$\tau$',\
          r'QKeras A $\rightarrow$ 4L', r'BP Keras A $\rightarrow$ 4L',\
          r'QKeras $h_{\pm} \rightarrow \tau\nu$', r'BP Keras $h_{\pm} \rightarrow \tau\nu$',\
          r'QKeras $h_{0} \rightarrow \tau\tau$', r'BP Keras $h_{0} \rightarrow \tau\tau$']
loss = '$D_{KL}$'

In [ ]:
for i, label in enumerate(bsm_labels):
    qkeras_pred = model.predict(bsm_data[i])
    BP_pred = BP_model.predict(bsm_data[i])
    
    kl_loss_total.append(qkeras_pred)
    kl_loss_total.append(BP_pred)
    print("========================================================================")

In [ ]:
# check range of loss distributions
minScore = 999999.
maxScore = 0
for i in range(len(labels)):
    thisMin = np.min(kl_loss_total[i])
    thisMax = np.max(kl_loss_total[i])
    minScore = min(thisMin, minScore)
    maxScore = max(maxScore, thisMax)


In [ ]:
colors = ['C1','C2', 'C3', 'C4', 'C5', 'C6']

In [ ]:
bin_size=100
plt.figure(figsize=(10,8))
z = 0
for i, label in enumerate(labels):
    if i%2==0:
        plt.hist(kl_loss_total[i].reshape(kl_loss_total[i].shape[0]*1), bins=bin_size, label=label, density = True, range=(minScore, 2),
         histtype='step', fill=False, linewidth=1.5, color=colors[z])
    if i%2==1:
        plt.hist(kl_loss_total[i].reshape(kl_loss_total[i].shape[0]*1), bins=bin_size, label=label, density = True, range=(minScore, 2),
         histtype='step', fill=False, linewidth=1.5, alpha=0.6, color=colors[z])
        z = z+1
#plt.semilogx()
plt.semilogy()
plt.xlabel("Loss")
plt.ylabel("Probability (a.u.)")
plt.grid(True)
plt.title('KL loss')
plt.legend(loc='best')
plt.show()

In [ ]:
from sklearn.metrics import roc_curve, auc
tpr_lq=[];fpr_lq=[];auc_lq=[]
tpr_ato4l=[];fpr_ato4l=[];auc_ato4l=[]
tpr_ch=[];fpr_ch=[];auc_ch=[]
tpr_to=[];fpr_to=[];auc_to=[]


target_qcd_qkeras = np.zeros(kl_loss_total[0].shape[0])
target_qcd_BP = np.zeros(kl_loss_total[1].shape[0])

for i, label in enumerate(labels):
    if i == 0 and i==1: continue
    if i%2==0:
        trueVal = np.concatenate((np.ones(kl_loss_total[i].shape[0]), target_qcd_qkeras))
        predVal_loss = np.concatenate((kl_loss_total[i], kl_loss_total[0]))

        fpr_loss, tpr_loss, threshold_loss = roc_curve(trueVal, predVal_loss)

        auc_loss = auc(fpr_loss, tpr_loss)
        if i==2:
            tpr_lq.append(tpr_loss)
            fpr_lq.append(fpr_loss)
            auc_lq.append(auc_loss)
        elif i == 4:
            tpr_ato4l.append(tpr_loss)
            fpr_ato4l.append(fpr_loss)
            auc_ato4l.append(auc_loss)
        elif i==6:
            tpr_ch.append(tpr_loss)
            fpr_ch.append(fpr_loss)
            auc_ch.append(auc_loss)
        elif i == 8:
            tpr_to.append(tpr_loss)
            fpr_to.append(fpr_loss)
            auc_to.append(auc_loss)
    if i%2==1:
        
        trueVal = np.concatenate((np.ones(kl_loss_total[i].shape[0]), target_qcd_BP))
        predVal_loss = np.concatenate((kl_loss_total[i], kl_loss_total[1]))

        fpr_loss, tpr_loss, threshold_loss = roc_curve(trueVal, predVal_loss)

        auc_loss = auc(fpr_loss, tpr_loss)
        if i==3:
            tpr_lq.append(tpr_loss)
            fpr_lq.append(fpr_loss)
            auc_lq.append(auc_loss)
        elif i == 5:
            tpr_ato4l.append(tpr_loss)
            fpr_ato4l.append(fpr_loss)
            auc_ato4l.append(auc_loss)
        elif i==7:
            tpr_ch.append(tpr_loss)
            fpr_ch.append(fpr_loss)
            auc_ch.append(auc_loss)
        elif i == 9:
            tpr_to.append(tpr_loss)
            fpr_to.append(fpr_loss)
            auc_to.append(auc_loss)

In [ ]:
plt.figure(figsize=(12,8))
for i, (tpr, fpr, auc, L) in enumerate(zip(tpr_lq[:], fpr_lq[:], auc_lq[:], labels[2:4])):
    if i == 1:
        plt.plot(fpr, tpr, "-", label='%s (auc = %.1f%%)'%(L,auc*100.), linewidth=1.5, color=colors[0], alpha=0.6)
    else: 
        plt.plot(fpr, tpr, "-", label='%s (auc = %.1f%%)'%(L,auc*100.), linewidth=1.5, color=colors[0])

for i, (tpr, fpr, auc, L) in enumerate(zip(tpr_ato4l[:], fpr_ato4l[:], auc_ato4l[:], labels[4:6])):
    if i == 1: plt.plot(fpr, tpr, "-", label='%s (auc = %.1f%%)'%(L,auc*100.), linewidth=1.5, color=colors[1], alpha = 0.6)
    else: plt.plot(fpr, tpr, "-", label='%s (auc = %.1f%%)'%(L,auc*100.), linewidth=1.5, color=colors[1])
for i, (tpr, fpr, auc, L) in enumerate(zip(tpr_ch[:], fpr_ch[:], auc_ch[:], labels[6:8])):
    if i==1: plt.plot(fpr, tpr, "-", label='%s (auc = %.1f%%)'%(L,auc*100.), linewidth=1.5, color=colors[2], alpha=0.6)
    else: plt.plot(fpr, tpr, "-", label='%s (auc = %.1f%%)'%(L,auc*100.), linewidth=1.5, color=colors[2])

for i, (tpr, fpr, auc, L) in enumerate(zip(tpr_to[:], fpr_to[:], auc_to[:], labels[8:])):
    if i==1: plt.plot(fpr, tpr, "-", label='%s (auc = %.1f%%)'%(L,auc*100.), linewidth=1.5, color=colors[3], alpha=0.6)
    else: plt.plot(fpr, tpr, "-", label='%s (auc = %.1f%%)'%(L,auc*100.), linewidth=1.5, color=colors[3])
plt.semilogx()
plt.semilogy()
plt.ylabel("True Positive Rate", fontsize=15)
plt.xlabel("False Positive Rate", fontsize=15)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.grid(True)
#plt.ylim(0,0.5)
plt.legend(bbox_to_anchor=[1.2, 0.5],loc='best',frameon=True)
plt.tight_layout()
plt.plot(np.linspace(0, 1),np.linspace(0, 1), '--', color='0.75')
plt.axvline(0.00001, color='red', linestyle='dashed', linewidth=1)
#plt.title('QKERAS <16,6>')
plt.show()

In [ ]:
output_result = '/eos/user/e/epuljak/forDelphes/CorrectDataResults/PTQ_VAE_result_qkeras2_1610.h5'

In [ ]:
h5f = h5py.File(output_result, 'w')
h5f.create_dataset('QCD_Qkeras', data = kl_loss_total[0])
h5f.create_dataset('QCD_BP', data = kl_loss_total[1])
for i,bsm in enumerate(bsm_labels[:]):
    print(i)
    if i == 0: z = 2
    elif i == 1: z = 4
    elif i == 2: z = 6
    elif i == 3: z = 8
    h5f.create_dataset('%s_Qkeras' %bsm, data = kl_loss_total[z])
    h5f.create_dataset('%s_BP'%bsm, data = kl_loss_total[z+1])
h5f.close()